In [1]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

from cProfile import label

%matplotlib inline

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
directory_gsat = 'gsat2_results'
directory_probsat = 'probsat_results'

In [4]:
directory_gsat_20 = 'gsat2_results/results_20'
directory_gsat_50 = 'gsat2_results/results_50'
directory_gsat_70 = 'gsat2_results/results_70'

In [5]:
directory_probsat_20 = 'probsat_results/results_20'
directory_probsat_50 = 'probsat_results/results_50'
directory_probsat_70 = 'probsat_results/results_70'

In [6]:
directories_gsat = [directory_gsat_20, directory_gsat_50, directory_gsat_70]
directories_probsat = [directory_probsat_20, directory_probsat_50, directory_probsat_70]

In [29]:
probsat_20 = pd.DataFrame(columns=['inst', 'succ', 'steps', 'max_iter', 'awg fined', 'mu', 'sig^2']) 
probsat_50 = pd.DataFrame(columns=['inst', 'succ', 'steps', 'max_iter', 'awg fined', 'mu', 'sig^2']) 
probsat_70 = pd.DataFrame(columns=['inst', 'succ', 'steps', 'max_iter', 'awg fined', 'mu', 'sig^2']) 

In [30]:
gsat_20 = pd.DataFrame(columns=['inst', 'succ', 'steps', 'max_iter', 'awg fined', 'mu', 'sig^2']) 
gsat_50 = pd.DataFrame(columns=['inst', 'succ', 'steps', 'max_iter', 'awg fined', 'mu', 'sig^2']) 
gsat_70 = pd.DataFrame(columns=['inst', 'succ', 'steps', 'max_iter', 'awg fined', 'mu', 'sig^2']) 

In [34]:
def fill_df(df_to_fill, directory):
    for filename in os.scandir(directory):
        if filename.is_file():
            d = pd.read_csv(filename.path, delimiter=' ')
            if(len(d.columns) != 4):
                print(filename.name)
                print(d)
                break
            df = pd.read_csv(filename.path, delimiter=' ',names=['iter', 'max_iter', 'sat_clauses','max_sat_clauses'])
            n = len(df)
            
            for i in range(n):
                if df.loc[i, 'iter'] == 0:
                    df.loc[i, 'iter'] = 1
            
            df_to_fill.loc[len(df_to_fill.index)] = [filename.name.split('.')[0], np.nan, np.nan, 3000, np.nan, np.nan, np.nan]
                
            # succ probsat
            df_to_fill.loc[df_to_fill['inst'] == filename.name.split('.')[0], 
                   'succ'] = len(df[df['iter'] != df['max_iter']])

            # steps probsat
            df_to_fill.loc[df_to_fill['inst'] == filename.name.split('.')[0], 
                   'steps'] = df['iter'].mean()

            # awg fined probsat
            n = len(df)
            sm = 0
            for i in range(n):
                if df['max_iter'][i] == df.loc[i, 'iter']:
                    sm += 10 * df.loc[i, 'iter']
                else:
                    sm += df.loc[i, 'iter']
            fined_awg = sm / n
            df_to_fill.loc[df_to_fill['inst'] == filename.name.split('.')[0], 
                   'awg fined'] = fined_awg

            # mu probsat
            iters = np.log(df['iter'])
            mu = sum(iters) / n
            df_to_fill.loc[df_to_fill['inst'] == filename.name.split('.')[0], 
                   'mu'] = mu

            # sig^2 probsat
            sigma2 = np.sum([(x_i - mu) ** 2 for x_i in iters]) / (n - 1)
            df_to_fill.loc[df_to_fill['inst'] == filename.name.split('.')[0], 
                   'sig^2'] = sigma2

In [35]:
fill_df(probsat_20, directory_probsat_20)

In [36]:
probsat_20

,inst,succ,steps,max_iter,awg fined,mu,sig^2
0,ruf20-551,1000.0,41.559,NaN,41.559,3.289269,0.870453
1,ruf20-73,1000.0,136.458,NaN,136.458,4.446896,1.118555
2,ruf20-368,1000.0,98.229,NaN,98.229,4.154754,0.978792
3,ruf20-165,1000.0,30.594,NaN,30.594,3.143808,0.554245
4,ruf20-713,1000.0,47.070,NaN,47.070,3.363232,0.859760
5,ruf20-843,998.0,414.619,NaN,468.619,5.354627,1.860818
6,ruf20-343,1000.0,115.537,NaN,115.537,4.319880,0.952808
7,ruf20-923,1000.0,66.960,NaN,66.960,3.903195,0.660008
8,ruf20-957,1000.0,82.767,NaN,82.767,3.986313,0.927308
9,ruf20-918,1000.0,165.879,NaN,165.879,4.516048,1.396790


In [37]:
fill_df(probsat_50, directory_probsat_50)

In [38]:
probsat_50

,inst,succ,steps,max_iter,awg fined,mu,sig^2
0,ruf50-835,1000.0,218.703,3000,218.703,5.109488,0.571325
1,ruf50-366,999.0,425.117,3000,452.117,5.642615,0.897724
2,ruf50-816,999.0,324.862,3000,351.862,5.359001,0.871206
3,ruf50-877,988.0,782.763,3000,1106.763,6.244250,1.021754
4,ruf50-776,1000.0,377.636,3000,377.636,5.517857,0.947230
5,ruf50-487,1000.0,166.934,3000,166.934,4.807115,0.623108
6,ruf50-766,750.0,1546.535,3000,8296.535,6.908812,1.288102
7,ruf50-995,1000.0,224.762,3000,224.762,5.018743,0.803635
8,ruf50-127,993.0,458.251,3000,647.251,5.624076,1.034754
9,ruf50-736,910.0,1089.990,3000,3519.990,6.491574,1.329240


In [39]:
fill_df(probsat_70, directory_probsat_70)

In [40]:
probsat_70

,inst,succ,steps,max_iter,awg fined,mu,sig^2
0,ruf75-525,992.0,601.319,3000,817.319,6.013526,0.815612
1,ruf75-119,608.0,1893.656,3000,12477.656,7.226264,0.922641
2,ruf75-645,1000.0,463.605,3000,463.605,5.813638,0.676964
3,ruf75-4,863.0,1370.254,3000,5069.254,6.918383,0.742993
4,ruf75-679,988.0,689.991,3000,1013.991,6.167844,0.789314
5,ruf75-949,999.0,559.015,3000,586.015,6.028775,0.620330
6,ruf75-543,430.0,2191.295,3000,17581.295,7.441754,0.767800
7,ruf75-324,995.0,548.035,3000,683.035,5.971014,0.678600
8,ruf75-281,940.0,1098.871,3000,2718.871,6.712103,0.667844
9,ruf75-819,1000.0,453.713,3000,453.713,5.815009,0.625958


In [41]:
fill_df(gsat_50, directory_gsat_50)

In [42]:
gsat_50

,inst,succ,steps,max_iter,awg fined,mu,sig^2
0,ruf50-835,996.0,346.589,3000,454.589,5.346139,0.964313
1,ruf50-366,966.0,756.745,3000,1674.745,6.086692,1.300167
2,ruf50-816,990.0,598.028,3000,868.028,5.836155,1.252020
3,ruf50-877,860.0,1242.468,3000,5022.468,6.664628,1.221710
4,ruf50-776,976.0,726.810,3000,1374.810,6.038612,1.381644
5,ruf50-487,1000.0,292.463,3000,292.463,5.212291,0.999147
6,ruf50-766,462.0,2124.361,3000,16650.361,7.298827,1.296206
7,ruf50-995,996.0,443.771,3000,551.771,5.518629,1.271861
8,ruf50-127,849.0,1040.901,3000,5117.901,6.253733,1.717135
9,ruf50-736,675.0,1688.068,3000,10463.068,6.956643,1.507719


In [43]:
fill_df(gsat_20, directory_gsat_20)

In [44]:
gsat_20

,inst,succ,steps,max_iter,awg fined,mu,sig^2
0,ruf20-551,1000.0,52.520,3000,52.520,3.515600,0.936389
1,ruf20-73,1000.0,194.329,3000,194.329,4.614462,1.497328
2,ruf20-368,1000.0,174.001,3000,174.001,4.460857,1.648670
3,ruf20-165,1000.0,34.334,3000,34.334,3.128601,0.735732
4,ruf20-713,1000.0,54.794,3000,54.794,3.367686,1.085163
5,ruf20-843,956.0,824.385,3000,2012.385,5.836128,2.753849
6,ruf20-343,1000.0,171.548,3000,171.548,4.538624,1.343557
7,ruf20-923,1000.0,96.903,3000,96.903,3.972874,1.110490
8,ruf20-957,1000.0,125.500,3000,125.500,4.145951,1.338165
9,ruf20-918,1000.0,285.787,3000,285.787,4.843934,2.089085


In [45]:
fill_df(gsat_70, directory_gsat_70)

In [46]:
gsat_70

,inst,succ,steps,max_iter,awg fined,mu,sig^2
0,ruf75-525,882.0,1111.734,3000,4297.734,6.532360,1.163557
1,ruf75-119,306.0,2422.063,3000,21160.063,7.582359,0.704320
2,ruf75-645,911.0,1004.402,3000,3407.402,6.417633,1.177979
3,ruf75-4,602.0,1980.318,3000,12726.318,7.341047,0.715411
4,ruf75-679,904.0,1039.179,3000,3631.179,6.520134,0.976592
5,ruf75-949,943.0,975.328,3000,2514.328,6.493704,0.894876
6,ruf75-543,284.0,2477.267,3000,21809.267,7.642603,0.570730
7,ruf75-324,861.0,1015.681,3000,4768.681,6.435661,1.088155
8,ruf75-281,697.0,1779.014,3000,9960.014,7.204184,0.768916
9,ruf75-819,932.0,878.325,3000,2714.325,6.251464,1.190901


In [48]:
gsat_20.to_csv('gsat_20.csv',index=False)
gsat_50.to_csv('gsat_50.csv',index=False)
gsat_70.to_csv('gsat_70.csv',index=False)

probsat_20.to_csv('probsat_20.csv',index=False)
probsat_50.to_csv('probsat_50.csv',index=False)
probsat_70.to_csv('probsat_70.csv',index=False)